In [8]:
import pandas as pd
import numpy as np

# 加载数据
sf_permits = pd.read_csv("../data/Building_Permits.csv")

np.random.seed(0)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#1.观察数据
sf_permits.sample(10) #随机抽取十条观察 可知很多数据项为NaN，这些就是我们需要处理的数据

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
40553,201403039652,8,otc alterations permit,03/03/2014,3732,008,400,NaN,Clementina,St,...,NaN,NaN,1.0,constr type 1,NaN,6.0,South of Market,94103.0,"(37.780460571778164, -122.40450626524974)",1334094491645
169731,201510159735,3,additions alterations or repairs,10/15/2015,2609,028,79,NaN,Buena Vista,Tr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94117.0,"(37.76757916496494, -122.43793170417105)",1399356139170
19180,M409787,8,otc alterations permit,07/22/2013,4624,031,178,NaN,West Point,Rd,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73524725436046, -122.38063828309745)",1311685491725
68047,201411191888,8,otc alterations permit,11/19/2014,0039,109,294,NaN,Francisco,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.805257822817126, -122.40998545760392)",1362881288870
64238,M527228,8,otc alterations permit,10/14/2014,1251,002,707,NaN,Cole,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Haight Ashbury,94117.0,"(37.76836885973765, -122.45074431487859)",135886493776
27703,M434527,8,otc alterations permit,10/11/2013,0348,015,277,NaN,Golden Gate,Av,...,NaN,NaN,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.781433365757565, -122.41487518938789)",1320626301305
109281,201512175360,8,otc alterations permit,12/17/2015,0061,013,40,NaN,Whiting,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.80389212201969, -122.40804427557475)",140688357279
15075,201306109020,8,otc alterations permit,06/10/2013,0262,020,100,NaN,Pine,St,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Financial District/South Beach,94111.0,"(37.792635570892635, -122.39899511726274)",1307390420381
59124,201408264765,8,otc alterations permit,08/26/2014,0321,016,650,NaN,Ellis,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,6.0,Tenderloin,94109.0,"(37.78461816669083, -122.41700445197698)",1353527233881
175167,201702240086,8,otc alterations permit,02/24/2017,6533,028,451,NaN,Fair Oaks,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Mission,94110.0,"(37.74948705332759, -122.42343069698013)",1454274206536


In [10]:
#2.观察缺失值的数量
missing_values_count = sf_permits.isnull().sum()
missing_values_count[0:15]

Permit Number                  0
Permit Type                    0
Permit Type Definition         0
Permit Creation Date           0
Block                          0
Lot                            0
Street Number                  0
Street Number Suffix      196684
Street Name                    0
Street Suffix               2768
Unit                      169421
Unit Suffix               196939
Description                  290
Current Status                 0
Current Status Date            0
dtype: int64

In [11]:
total_cells = np.product(sf_permits.shape)
total_missings = missing_values_count.sum()

(total_missings / total_cells) * 100 #查看缺失值占总数的百分比

26.26002315058403

In [12]:
#3.分析出现缺失值的原因 （未被记录 or 不存在）
#若不存在，则继续作为NaN， 若未被记录则猜测其可能值补充

# 查看Street Number Suffix 和 Zipcode 的缺失值

missing_values_count[["Street Number Suffix", "Zipcode"]] #SNS下有大量缺失值，可能根本不存在。Zipcode(邮编)可能为缺失

Street Number Suffix    196684
Zipcode                   1716
dtype: int64

In [13]:
#4.剔除缺失值
#方案一：直接剔除任何包含缺失值的行或列（不推荐）
sf_permits.dropna() #得到零行（每行都有缺失值）


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


In [14]:
columns_with_na_dropped = sf_permits.dropna(axis=1)
columns_with_na_dropped.head()#43列变成了12列

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [15]:
#5.自动补全缺失值
subset_sf_permits = sf_permits.loc[:,"Street Number Suffix":"Issued Date"].head() #截取一部分观察
subset_sf_permits

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date
0,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017
2,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017


In [16]:
# 如果数据的类型都是数字，我们可以考虑把所有的缺失值都填为 0:
subset_sf_permits.fillna(0)

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date
0,0,Ellis,St,0.0,0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015
1,0,Geary,St,0.0,0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017
2,0,Pacific,Av,0.0,0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,0
3,0,Pacific,Av,0.0,0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017
4,0,Market,St,0.0,0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017


In [17]:
# string 型的数据，还有另一种选择就是将空值置为与它相邻的下一行对应的数据，没有下一行数据就置为 0
subset_sf_permits.fillna(method = "bfill", axis=0).fillna("0")

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date
0,0,Ellis,St,0,0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015
1,0,Geary,St,0,0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017
2,0,Pacific,Av,0,0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017
3,0,Pacific,Av,0,0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017
4,0,Market,St,0,0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017
